# Chapter 18: Reinforcement Learning

Reinforcement Learning (RL) adalah bidang Machine Learning di mana sebuah **Agen** belajar berinteraksi dalam sebuah **Lingkungan** (*Environment*). Agen melakukan **Aksi** (*Action*) dan menerima **Imbalan** (*Reward*) atau penalti. Tujuannya adalah untuk mempelajari **Kebijakan** (*Policy*) optimal yang memaksimalkan akumulasi imbalan seiring waktu.

### Konsep Utama:
1. **Agen & Lingkungan**: Entitas yang belajar dan dunia tempat ia beraksi.
2. **Aksi, State, & Reward**: Apa yang dilakukan agen, kondisi saat ini, dan umpan balik yang diterima.
3. **Policy (Kebijakan)**: Algoritma yang digunakan agen untuk menentukan aksi berdasarkan state.
4. **OpenAI Gym**: Toolkit standar untuk mengembangkan dan membandingkan algoritma RL.

## 1. Menyiapkan Lingkungan dengan OpenAI Gym

Kita akan menggunakan masalah klasik **CartPole**, di mana agen harus menjaga tiang tetap tegak dengan menggerakkan kereta ke kiri atau ke kanan.

In [ ]:
import gym
import numpy as np

# Membuat lingkungan CartPole
env = gym.make("CartPole-v1")

# Inisialisasi lingkungan
obs = env.reset()
print("Observasi awal (State):", obs)

# Contoh melakukan satu langkah aksi secara acak
action = 1 # 0 untuk kiri, 1 untuk kanan
obs, reward, done, info = env.step(action)

print(f"Observasi Baru: {obs}")
print(f"Reward: {reward}")
print(f"Selesai (Done): {done}")

## 2. Neural Network Policies

Alih-alih menggunakan aturan keras (*hard-coded*), kita bisa menggunakan Neural Network yang menerima observasi sebagai input dan mengeluarkan probabilitas untuk setiap aksi.

In [ ]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 4 # Observasi CartPole memiliki 4 fitur

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="relu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid") # Output probabilitas untuk aksi '1'
])

print("Model Kebijakan (Policy Model) berhasil dibangun.")

## 3. Algoritma Policy Gradients (REINFORCE)

Policy Gradients melatih agen dengan mengevaluasi hasil dari seluruh episode. Aksi yang menghasilkan total reward tinggi akan diperkuat (probabilitasnya ditingkatkan), sedangkan aksi yang buruk akan diperlemah.

### Langkah Pelatihan:
1. Biarkan agen bermain beberapa episode.
2. Hitung total reward untuk setiap langkah (dengan *discount factor*).
3. Lakukan Backpropagation untuk menyesuaikan bobot model.

In [ ]:
def discount_rewards(rewards, discount_factor):
    # Menghitung imbalan masa depan yang didiskon
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(r, discount_factor) for r in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(r - reward_mean) / reward_std for r in all_discounted_rewards]

print("Fungsi pemrosesan reward siap digunakan.")

## 4. Q-Learning & Markov Decision Processes (MDP)

Q-Learning adalah algoritma berbasis nilai yang mencoba mempelajari **Q-Value** (kualitas) dari setiap pasangan (State, Action).

### Persamaan Bellman:
$$Q(s, a) \leftarrow (1 - \alpha)Q(s, a) + \alpha(r + \gamma \max_{a'} Q(s', a'))$$

Di mana:
- $\alpha$ adalah *learning rate*.
- $\gamma$ adalah *discount factor*.

## 5. Deep Q-Learning (DQN)

Untuk lingkungan dengan state space yang besar, kita tidak bisa menggunakan tabel. Sebagai gantinya, kita menggunakan Neural Network untuk mengestimasi Q-Values.

### Teknik Kritis dalam DQN:
1. **Experience Replay**: Menyimpan pengalaman lama dan mengambil sampel acak untuk pelatihan (mengurangi korelasi data).
2. **Target Network**: Menggunakan jaringan kedua yang jarang diperbarui untuk menstabilkan target prediksi Q-Value.

In [ ]:
from collections import deque

replay_buffer = deque(maxlen=2000)

def epsilon_greedy_policy(state, epsilon=0):
    # Eksplorasi vs Eksploitasi
    if np.random.rand() < epsilon:
        return np.random.randint(2)
    else:
        Q_values = model.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

print("Komponen dasar Deep Q-Learning siap.")

## Kesimpulan Praktis

1. **Reinforcement Learning** sangat berbeda dari supervised learning karena target (imbalan) seringkali tertunda (*delayed rewards*).
2. **Policy Gradients** efektif untuk aksi kontinu dan ruang observasi besar.
3. **DQN** adalah pondasi untuk banyak kesuksesan RL modern (seperti agen Atari), tetapi memerlukan teknik seperti *Experience Replay* untuk stabilitas.
4. **TF-Agents** adalah library TensorFlow yang sangat direkomendasikan untuk implementasi RL tingkat produksi di dunia nyata.